## Using CNN and NCM for MNIST feature extraction and classification
### Summary
MNIST 데이터셋을 분류하기 위해 먼저 CNN을 Pretraining 시킨 후, FC layer만 제거하여 Feature Extractor로 활용.

Feature Extractor로써 CNN만 추가한 것과 PCA or LDA를 추가하여 NCM을 실험한 결과들.

## Table of Contents

* [Load MNIST Dataset](#load-mnist-dataset)
* [Pretraining CNN](#pretraining-cnn)
* [64 Feature Extractor (CNN)](#64-feature-extrator-cnn)
* [CNN feature + NCM](#cnn-feature--ncm)
* [CNN feature + PCA + NCM](#cnn-feature--pca--ncm)
  * [PCA run function](#pca-run-function)
  * [CNN features - 2 PCA - NCM](#cnn-features---2-pca---ncm)
  * [CNN features - 3 PCA - NCM](#cnn-features---3-pca---ncm)
  * [CNN features - 5 PCA - NCM](#cnn-features---5-pca---ncm)
  * [CNN features - 26 PCA - NCM](#cnn-features---26-pca---ncm)
  * [CNN features - 43 PCA - NCM](#cnn-features---43-pca---ncm)
* [CNN feature + LDA + NCM](#cnn-feature--lda--ncm)
  * [LDA run function](#lda-run-function)
  * [CNN features - 2 LDA - NCM](#cnn-features---2-lda---ncm)
  * [CNN features - 3 LDA - NCM](#cnn-features---3-lda---ncm)
  * [CNN features - 5 LDA - NCM](#cnn-features---5-lda---ncm)
  * [CNN features - 9 LDA - NCM](#cnn-features---9-lda---ncm)

In [1]:
import time
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn

from utils.utils import Info, accuracy

from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import NearestCentroid

import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
# from utils.NCM_Classifier import train, validate
# from models.resnet_feature import resnet18_feature

In [2]:
class Config(Info):
    def __init__(self):
        super(Info, self).__init__()
        self.device = 'PC'
        self.dataset = 'MNIST'
        self.test_size = 0.2
        self.feature_size = 784
        self.method = 'NCM'
        self.distance = 'Euclidean'
        self.reduction_method = [None, None] # method, n_components
        self.iter = 10

In [3]:
config = Config()
config.info()
config.print_rutin()

Device ── PC
│
├──Dataset
│    └────MNIST
│    └────Train size 80%
│    └────Feature size: 784
│
├──Method
│    └────NCM
│    └────Euclidean
│
├──Dimension reduction
│    └────Method: None
│    └────Component size: None
│    └────Feature Reduction Ratio: None%
│
└──Iteration
    └────10
PC - MNIST(80%) - NCM - 10 iteration


In [4]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed) # if use multi-GPU
cudnn.deterministic = True  # 연산 처리 속도 감소 -> 모델과 코드를 배포해야 하는 연구 후반 단계에 사용
cudnn.benchmark = False

## Load MNIST Dataset

In [5]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))
                                ])

batch_size = 512

trainset = torchvision.datasets.MNIST(root='../../datasets', train=True, download=False, transform=transform)
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

validationset = torchvision.datasets.MNIST(root='../../datasets', train=False, download=False, transform=transform)
val_loader = DataLoader(validationset, batch_size=batch_size, shuffle=False, num_workers=2)

print(trainset.data.shape)
print(validationset.data.shape)

torch.Size([60000, 28, 28])
torch.Size([10000, 28, 28])


c:\Users\Cau_13\Anaconda3\envs\tf-venv\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## Pretraining CNN

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.conv3 = nn.Conv2d(32, 64, 3)

        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.relu = nn.ReLU(inplace=True)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, 10)

    def forward(self, x): # 28 x 28
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x) # 14 x 14

        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x) # 7 x 7

        x = self.conv3(x)
        x = self.relu(x)
        x = self.maxpool(x) # 4 x 4

        x = self.avgpool(x) # 1 x 1
        x = torch.flatten(x, 1)
        x = self.fc(x)
        
        return x

In [7]:
learning_rate = 0.001
training_epochs = 20

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

net = CNN().to(device)

criterion = torch.nn.CrossEntropyLoss()  # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

total_train_batch = len(train_loader)
total_test_batch = len(val_loader)
print('총 배치의 수 : Train-{}, Test-{}'.format(total_train_batch, total_test_batch))
net

총 배치의 수 : Train-118, Test-20


CNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (relu): ReLU(inplace=True)
  (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc): Linear(in_features=64, out_features=10, bias=True)
)

In [8]:
for epoch in range(training_epochs):
    avg_cost = 0
    avg_train_acc = 0
    avg_test_acc = 0

    net.train()
    for X, Y in train_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y느 ㄴ레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = net(X)
        cost = criterion(hypothesis, Y)
        acc = accuracy(hypothesis, Y)

        cost.backward()
        optimizer.step()

        avg_cost += cost / total_train_batch
        avg_train_acc += acc / total_train_batch

    print('[Epoch: {:>4}] Acc : {:>4} cost : {:>.9}'.format(epoch + 1, avg_train_acc.item(), avg_cost.item()))

    net.eval()
    with torch.no_grad():
        for X_test, Y_test in val_loader:
            X_test = X_test.to(device)
            Y_test = Y_test.to(device)

            prediction = net(X_test)

            acc = accuracy(prediction, Y_test)
            avg_test_acc += acc / total_test_batch

    print('Test Acc: {:>4}'.format(avg_test_acc.item()) )

[Epoch:    1] Acc : 76.8609848022461 cost : 0.881799221
Test Acc: 94.0647964477539
[Epoch:    2] Acc : 94.56600189208984 cost : 0.183034062
Test Acc: 96.19657897949219
[Epoch:    3] Acc : 96.1384506225586 cost : 0.128635019
Test Acc: 96.94622802734375
[Epoch:    4] Acc : 96.85572814941406 cost : 0.105369374
Test Acc: 96.46771240234375
[Epoch:    5] Acc : 97.2474365234375 cost : 0.0903387964
Test Acc: 98.08306884765625
[Epoch:    6] Acc : 97.62645721435547 cost : 0.0771759674
Test Acc: 97.93658447265625
[Epoch:    7] Acc : 97.84662628173828 cost : 0.0704659894
Test Acc: 98.34558868408203
[Epoch:    8] Acc : 98.05178833007812 cost : 0.0641076788
Test Acc: 98.35650634765625
[Epoch:    9] Acc : 98.21627044677734 cost : 0.0577964783
Test Acc: 98.41165161132812
[Epoch:   10] Acc : 98.2978515625 cost : 0.0547632203
Test Acc: 98.32605743408203
[Epoch:   11] Acc : 98.52849578857422 cost : 0.0497457087
Test Acc: 98.66670227050781
[Epoch:   12] Acc : 98.55992889404297 cost : 0.0472592115
Test Acc

## 64 Feature Extrator (CNN)

In [9]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

net.fc = Identity()
net

CNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (relu): ReLU(inplace=True)
  (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc): Identity()
)

## CNN feature + NCM

### CNN feature extract function

In [14]:
def extract_cnn_feature(net, 
                       train_loader, 
                       val_loader):

    def extract_feature(loader):
        features = []
        targets = []

        start_time = time.time()
        for i, (images, target) in enumerate(loader):
            if torch.cuda.is_available():
                images = images.to(device)
                target = target.to(device)

            # compute output
            feature = net(images)
            features.extend(feature.cpu().numpy())
            targets.extend(target.cpu().numpy())
        feature_extract_time = time.time() - start_time
        features = np.array(features)
        targets = np.array(targets)

        return features, targets, feature_extract_time
    
    net.eval()

    with torch.no_grad():
        # Train dataset
        train_features, train_targets, feature_extract_time = extract_feature(train_loader)
        # Test dataset
        test_features, test_targets, test_feature_extract_time = extract_feature(val_loader)
        
    return train_features, train_targets, test_features, test_targets, feature_extract_time, test_feature_extract_time
    


In [15]:
train_features, train_targets, test_features, test_targets, \
feature_extract_time, test_feature_extract_time = extract_cnn_feature(net, 
                                                                      train_loader=train_loader, 
                                                                      val_loader=val_loader)

### CNN + NCM run function

In [16]:
def cnn_ncm_run(train_features,
            train_targets,
            test_features,
            test_targets,
            config = None,
            metric = 'euclidean'):

    classifier = NearestCentroid(metric=metric)

    # Train dataset
    c_start_time = time.time()
    classifier.fit(train_features, train_targets)
    fit_time = time.time() - c_start_time
    output = classifier.predict(train_features)

    # measure accuracy
    acc = accuracy_score(output, train_targets)

    # Test dataset
    c_start_time = time.time()
    output = classifier.predict(test_features)
    predict_time = time.time() - c_start_time

    # measure accuracy
    test_acc = accuracy_score(output, test_targets)

    print('\nFinished Fit\n')
    print("Train Data Feature Extract Time : %.4f" % feature_extract_time, "sec")
    print("Train Data Fitting NCM Time : %.4f" % fit_time, "sec")
    print("Train Accuracy : %.2f" % (acc*100), "%")
    print('\nFinished Predicting\n')
    print("Test Data Feature Extract Time : %.4f" % test_feature_extract_time, "sec")
    print("Test Data Prediction Time : %.4f" % predict_time, "sec")
    print("Test Accuracy : %.2f" % (test_acc*100), "%")

    return acc, test_acc, feature_extract_time, test_feature_extract_time

In [17]:
config.info()

Device ── PC
│
├──Dataset
│    └────MNIST
│    └────Train size 80%
│    └────Feature size: 784
│
├──Method
│    └────NCM
│    └────Euclidean
│
├──Dimension reduction
│    └────Method: None
│    └────Component size: None
│    └────Feature Reduction Ratio: None%
│
└──Iteration
    └────10


In [18]:
acc, test_acc, feature_extract_time, test_feature_extract_time = \
    cnn_ncm_run(train_features, 
                train_targets, 
                test_features, 
                test_targets,
                config = config,
                metric = 'euclidean')


Finished Fit

Train Data Feature Extract Time : 4.2202 sec
Train Data Fitting NCM Time : 0.0140 sec
Train Accuracy : 94.62 %

Finished Predicting

Test Data Feature Extract Time : 1.3960 sec
Test Data Prediction Time : 0.0040 sec
Test Accuracy : 95.39 %


# CNN feature + PCA + NCM

### PCA run function

In [19]:
def pca_run(train_data_X,
            test_data_X,
            config=None):

    assert config is not None
#     config.info()

    n_components = config.reduction_method[1]

    pca_dims = PCA(n_components)
    print(f"The number of components : {n_components}")

    start_time = time.perf_counter()
    pca_dims.fit(train_data_X)
    pca_fit_time = time.perf_counter () - start_time
    print()
    print(f"Calculating SVD Matrix Time on Train Data-{train_data_X.shape} : {pca_fit_time:4f} sec")
    
    start_time = time.perf_counter()
    train_features = pca_dims.transform(train_data_X)
    train_features_extract_time = time.perf_counter () - start_time
    print(f"Transform train X-{train_data_X.shape} to {n_components}-PCA Time: {train_features_extract_time:4f} sec")

    start_time = time.perf_counter()
    test_features = pca_dims.transform(test_data_X)
    test_features_extract_time = time.perf_counter () - start_time
    print(f"Transform test X-{train_data_X.shape} to {n_components}-PCA Time: {test_features_extract_time:4f} sec")

    return pca_dims, train_features, test_features, pca_fit_time, train_features_extract_time, test_features_extract_time

### CNN features - 2 PCA - NCM

In [30]:
config.reduction_method = ['PCA', 2]
config.info()

Device ── PC
│
├──Dataset
│    └────MNIST
│    └────Train size 80%
│    └────Feature size: 784
│
├──Method
│    └────NCM
│    └────Euclidean
│
├──Dimension reduction
│    └────Method: PCA
│    └────Component size: 2
│    └────Feature Reduction Ratio: 0.3%
│
└──Iteration
    └────10


In [31]:
pca_dims, pca_train_features, pca_test_features, pca_fit_time, pca_train_features_extract_time, pca_test_features_extract_time = \
    pca_run(train_data_X = train_features,
            test_data_X = test_features,
            config=config)

The number of components : 2

Calculating SVD Matrix Time on Train Data-(60000, 64) : 0.057550 sec
Transform train X-(60000, 64) to 2-PCA Time: 0.007760 sec
Transform test X-(60000, 64) to 2-PCA Time: 0.001408 sec


In [32]:
acc, test_acc, feature_extract_time, test_feature_extract_time = \
    cnn_ncm_run(pca_train_features, 
                train_targets, 
                pca_test_features, 
                test_targets,
                config = config,
                metric = 'euclidean')


Finished Fit

Train Data Feature Extract Time : 4.2202 sec
Train Data Fitting NCM Time : 0.0060 sec
Train Accuracy : 55.73 %

Finished Predicting

Test Data Feature Extract Time : 1.3960 sec
Test Data Prediction Time : 0.0010 sec
Test Accuracy : 56.21 %


### CNN features - 3 PCA - NCM

In [33]:
config.reduction_method = ['PCA', 3]
config.info()

Device ── PC
│
├──Dataset
│    └────MNIST
│    └────Train size 80%
│    └────Feature size: 784
│
├──Method
│    └────NCM
│    └────Euclidean
│
├──Dimension reduction
│    └────Method: PCA
│    └────Component size: 3
│    └────Feature Reduction Ratio: 0.4%
│
└──Iteration
    └────10


In [34]:
pca_dims, pca_train_features, pca_test_features, pca_fit_time, pca_train_features_extract_time, pca_test_features_extract_time = \
    pca_run(train_data_X = train_features,
            test_data_X = test_features,
            config=config)

The number of components : 3

Calculating SVD Matrix Time on Train Data-(60000, 64) : 0.067082 sec
Transform train X-(60000, 64) to 3-PCA Time: 0.011019 sec
Transform test X-(60000, 64) to 3-PCA Time: 0.002021 sec


In [35]:
acc, test_acc, feature_extract_time, test_feature_extract_time = \
    cnn_ncm_run(pca_train_features, 
                train_targets, 
                pca_test_features, 
                test_targets,
                config = config,
                metric = 'euclidean')


Finished Fit

Train Data Feature Extract Time : 4.2202 sec
Train Data Fitting NCM Time : 0.0060 sec
Train Accuracy : 71.13 %

Finished Predicting

Test Data Feature Extract Time : 1.3960 sec
Test Data Prediction Time : 0.0020 sec
Test Accuracy : 72.16 %


### CNN features - 5 PCA - NCM

In [36]:
config.reduction_method = ['PCA', 5]
config.info()

Device ── PC
│
├──Dataset
│    └────MNIST
│    └────Train size 80%
│    └────Feature size: 784
│
├──Method
│    └────NCM
│    └────Euclidean
│
├──Dimension reduction
│    └────Method: PCA
│    └────Component size: 5
│    └────Feature Reduction Ratio: 0.6%
│
└──Iteration
    └────10


In [37]:
pca_dims, pca_train_features, pca_test_features, pca_fit_time, pca_train_features_extract_time, pca_test_features_extract_time = \
    pca_run(train_data_X = train_features,
            test_data_X = test_features,
            config=config)

The number of components : 5

Calculating SVD Matrix Time on Train Data-(60000, 64) : 0.076544 sec
Transform train X-(60000, 64) to 5-PCA Time: 0.007974 sec
Transform test X-(60000, 64) to 5-PCA Time: 0.002129 sec


In [38]:
acc, test_acc, feature_extract_time, test_feature_extract_time = \
    cnn_ncm_run(pca_train_features, 
                train_targets, 
                pca_test_features, 
                test_targets,
                config = config,
                metric = 'euclidean')


Finished Fit

Train Data Feature Extract Time : 4.2202 sec
Train Data Fitting NCM Time : 0.0050 sec
Train Accuracy : 85.73 %

Finished Predicting

Test Data Feature Extract Time : 1.3960 sec
Test Data Prediction Time : 0.0010 sec
Test Accuracy : 87.23 %


### CNN features - 26 PCA - NCM

In [39]:
config.reduction_method = ['PCA', 26]
config.info()

Device ── PC
│
├──Dataset
│    └────MNIST
│    └────Train size 80%
│    └────Feature size: 784
│
├──Method
│    └────NCM
│    └────Euclidean
│
├──Dimension reduction
│    └────Method: PCA
│    └────Component size: 26
│    └────Feature Reduction Ratio: 3.3000000000000003%
│
└──Iteration
    └────10


In [40]:
pca_dims, pca_train_features, pca_test_features, pca_fit_time, pca_train_features_extract_time, pca_test_features_extract_time = \
    pca_run(train_data_X = train_features,
            test_data_X = test_features,
            config=config)

The number of components : 26

Calculating SVD Matrix Time on Train Data-(60000, 64) : 0.103076 sec
Transform train X-(60000, 64) to 26-PCA Time: 0.012821 sec
Transform test X-(60000, 64) to 26-PCA Time: 0.002087 sec


In [41]:
acc, test_acc, feature_extract_time, test_feature_extract_time = \
    cnn_ncm_run(pca_train_features, 
                train_targets, 
                pca_test_features, 
                test_targets,
                config = config,
                metric = 'euclidean')


Finished Fit

Train Data Feature Extract Time : 4.2202 sec
Train Data Fitting NCM Time : 0.0080 sec
Train Accuracy : 94.62 %

Finished Predicting

Test Data Feature Extract Time : 1.3960 sec
Test Data Prediction Time : 0.0010 sec
Test Accuracy : 95.35 %


### CNN features - 43 PCA - NCM

In [42]:
config.reduction_method = ['PCA', 43]
config.info()

Device ── PC
│
├──Dataset
│    └────MNIST
│    └────Train size 80%
│    └────Feature size: 784
│
├──Method
│    └────NCM
│    └────Euclidean
│
├──Dimension reduction
│    └────Method: PCA
│    └────Component size: 43
│    └────Feature Reduction Ratio: 5.5%
│
└──Iteration
    └────10


In [43]:
pca_dims, pca_train_features, pca_test_features, pca_fit_time, pca_train_features_extract_time, pca_test_features_extract_time = \
    pca_run(train_data_X = train_features,
            test_data_X = test_features,
            config=config)

The number of components : 43

Calculating SVD Matrix Time on Train Data-(60000, 64) : 0.156589 sec
Transform train X-(60000, 64) to 43-PCA Time: 0.012767 sec
Transform test X-(60000, 64) to 43-PCA Time: 0.002439 sec


In [44]:
acc, test_acc, feature_extract_time, test_feature_extract_time = \
    cnn_ncm_run(pca_train_features, 
                train_targets, 
                pca_test_features, 
                test_targets,
                config = config,
                metric = 'euclidean')


Finished Fit

Train Data Feature Extract Time : 4.2202 sec
Train Data Fitting NCM Time : 0.0090 sec
Train Accuracy : 94.62 %

Finished Predicting

Test Data Feature Extract Time : 1.3960 sec
Test Data Prediction Time : 0.0020 sec
Test Accuracy : 95.36 %


# CNN feature + LDA + NCM

### LDA run function

In [45]:
def lda_run(train_data_X,
            train_data_y,
            test_data_X,
            config=None):

    assert config is not None
#     config.info()

    n_components = config.reduction_method[1]

    lda_dims = LinearDiscriminantAnalysis(n_components=n_components)
    print(f"The number of components : {n_components}")

    start_time = time.perf_counter()
    lda_dims.fit(train_data_X, train_data_y)
    lda_fit_time = time.perf_counter () - start_time
    print()
    print(f"Calculating LDA Matrix Time on Train Data-{train_data_X.shape} : {lda_fit_time:4f} sec")
    
    start_time = time.perf_counter()
    train_features = lda_dims.transform(train_data_X)
    train_features_extract_time = time.perf_counter () - start_time
    print(f"Transform train X-{train_data_X.shape} to {n_components}-LDA Time: {train_features_extract_time:4f} sec")

    start_time = time.perf_counter()
    test_features = lda_dims.transform(test_data_X)
    test_features_extract_time = time.perf_counter () - start_time
    print(f"Transform test X-{train_data_X.shape} to {n_components}-LDA Time: {test_features_extract_time:4f} sec")

    return lda_dims, train_features, test_features, lda_fit_time, train_features_extract_time, test_features_extract_time

### CNN features - 2 LDA - NCM

In [46]:
config.reduction_method = ['LDA', 2]
config.info()

Device ── PC
│
├──Dataset
│    └────MNIST
│    └────Train size 80%
│    └────Feature size: 784
│
├──Method
│    └────NCM
│    └────Euclidean
│
├──Dimension reduction
│    └────Method: LDA
│    └────Component size: 2
│    └────Feature Reduction Ratio: 0.3%
│
└──Iteration
    └────10


In [47]:
lda_dims, lda_train_features, lda_test_features, lda_fit_time, lda_train_features_extract_time, lda_test_features_extract_time = \
    lda_run(train_data_X = train_features,
            train_data_y = train_targets,
            test_data_X = test_features,
            config=config)

The number of components : 2

Calculating LDA Matrix Time on Train Data-(60000, 64) : 0.266032 sec
Transform train X-(60000, 64) to 2-LDA Time: 0.012951 sec
Transform test X-(60000, 64) to 2-LDA Time: 0.003570 sec


In [48]:
acc, test_acc, feature_extract_time, test_feature_extract_time = \
    cnn_ncm_run(lda_train_features, 
                train_targets, 
                lda_test_features, 
                test_targets,
                config = config)


Finished Fit

Train Data Feature Extract Time : 4.2202 sec
Train Data Fitting NCM Time : 0.0070 sec
Train Accuracy : 80.80 %

Finished Predicting

Test Data Feature Extract Time : 1.3960 sec
Test Data Prediction Time : 0.0010 sec
Test Accuracy : 81.45 %


### CNN features - 3 LDA - NCM

In [49]:
config.reduction_method = ['LDA', 3]
config.info()

Device ── PC
│
├──Dataset
│    └────MNIST
│    └────Train size 80%
│    └────Feature size: 784
│
├──Method
│    └────NCM
│    └────Euclidean
│
├──Dimension reduction
│    └────Method: LDA
│    └────Component size: 3
│    └────Feature Reduction Ratio: 0.4%
│
└──Iteration
    └────10


In [50]:
lda_dims, lda_train_features, lda_test_features, lda_fit_time, lda_train_features_extract_time, lda_test_features_extract_time = \
    lda_run(train_data_X = train_features,
            train_data_y = train_targets,
            test_data_X = test_features,
            config=config)

The number of components : 3

Calculating LDA Matrix Time on Train Data-(60000, 64) : 0.269271 sec
Transform train X-(60000, 64) to 3-LDA Time: 0.012475 sec
Transform test X-(60000, 64) to 3-LDA Time: 0.003209 sec


In [51]:
acc, test_acc, feature_extract_time, test_feature_extract_time = \
    cnn_ncm_run(lda_train_features, 
                train_targets, 
                lda_test_features, 
                test_targets,
                config = config)


Finished Fit

Train Data Feature Extract Time : 4.2202 sec
Train Data Fitting NCM Time : 0.0080 sec
Train Accuracy : 89.33 %

Finished Predicting

Test Data Feature Extract Time : 1.3960 sec
Test Data Prediction Time : 0.0010 sec
Test Accuracy : 89.49 %


### CNN features - 5 LDA - NCM

In [52]:
config.reduction_method = ['LDA', 5]
config.info()

Device ── PC
│
├──Dataset
│    └────MNIST
│    └────Train size 80%
│    └────Feature size: 784
│
├──Method
│    └────NCM
│    └────Euclidean
│
├──Dimension reduction
│    └────Method: LDA
│    └────Component size: 5
│    └────Feature Reduction Ratio: 0.6%
│
└──Iteration
    └────10


In [53]:
lda_dims, lda_train_features, lda_test_features, lda_fit_time, lda_train_features_extract_time, lda_test_features_extract_time = \
    lda_run(train_data_X = train_features,
            train_data_y = train_targets,
            test_data_X = test_features,
            config=config)

The number of components : 5

Calculating LDA Matrix Time on Train Data-(60000, 64) : 0.264045 sec
Transform train X-(60000, 64) to 5-LDA Time: 0.018373 sec
Transform test X-(60000, 64) to 5-LDA Time: 0.002920 sec


In [54]:
acc, test_acc, feature_extract_time, test_feature_extract_time = \
    cnn_ncm_run(lda_train_features, 
                train_targets, 
                lda_test_features, 
                test_targets,
                config = config)


Finished Fit

Train Data Feature Extract Time : 4.2202 sec
Train Data Fitting NCM Time : 0.0070 sec
Train Accuracy : 95.62 %

Finished Predicting

Test Data Feature Extract Time : 1.3960 sec
Test Data Prediction Time : 0.0010 sec
Test Accuracy : 95.59 %


### CNN features - 9 LDA - NCM

In [55]:
config.reduction_method = ['LDA', 9]
config.info()

Device ── PC
│
├──Dataset
│    └────MNIST
│    └────Train size 80%
│    └────Feature size: 784
│
├──Method
│    └────NCM
│    └────Euclidean
│
├──Dimension reduction
│    └────Method: LDA
│    └────Component size: 9
│    └────Feature Reduction Ratio: 1.0999999999999999%
│
└──Iteration
    └────10


In [56]:
lda_dims, lda_train_features, lda_test_features, lda_fit_time, lda_train_features_extract_time, lda_test_features_extract_time = \
    lda_run(train_data_X = train_features,
            train_data_y = train_targets,
            test_data_X = test_features,
            config=config)

The number of components : 9

Calculating LDA Matrix Time on Train Data-(60000, 64) : 0.267432 sec
Transform train X-(60000, 64) to 9-LDA Time: 0.011592 sec
Transform test X-(60000, 64) to 9-LDA Time: 0.002387 sec


In [57]:
acc, test_acc, feature_extract_time, test_feature_extract_time = \
    cnn_ncm_run(lda_train_features, 
                train_targets, 
                lda_test_features, 
                test_targets,
                config = config)


Finished Fit

Train Data Feature Extract Time : 4.2202 sec
Train Data Fitting NCM Time : 0.0080 sec
Train Accuracy : 98.28 %

Finished Predicting

Test Data Feature Extract Time : 1.3960 sec
Test Data Prediction Time : 0.0010 sec
Test Accuracy : 98.24 %
